In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np
import csv
import json

In [21]:
# read the csv file
data_years = [2016, 2017, 2018, 2019, 2020, 2021]
dfs = {}
counter = 1
for year in data_years:
    dfs[f'locations_{year}'] = pd.read_csv(f'{year}.csv', skiprows=3)
    dfs[f'locations_{year}'] = dfs[f'locations_{year}'].dropna()
    columns_to_be_removed = [1, 2, 3, 5, 6, 7, 8]
    dfs[f'locations_{year}'] = dfs[f'locations_{year}'].iloc[:, ~dfs[f'locations_{year}'].columns.isin(dfs[f'locations_{year}'].columns[columns_to_be_removed])]
    dfs[f'locations_{year}'].columns = ['State', 'Locations']
    value = dfs[f'locations_{year}']['Locations'].str.split('|', expand=True).astype(str)
    value[0] = value[0].str.replace(',', '').str.strip().astype(int)
    value[1] = value[1].str.replace(',', '').str.strip().astype(int)
    value.columns = [f'Number_of_Stations_{year}', f'Number_of_Outlets_{year}']
    dfs[f'locations_{year}'] = pd.concat([dfs[f'locations_{year}'], value], axis=1)
    dfs[f'locations_{year}'].drop(['Locations'], inplace = True, axis=1)
    if counter ==1:
        state_stations_df = dfs[f'locations_{year}']
        counter = counter + 1
    else:
        state_stations_df = pd.merge(state_stations_df, dfs[f'locations_{year}'], on = 'State')
state_stations_df = state_stations_df.set_index(state_stations_df.columns[0])
state_stations_df = state_stations_df.rename(index = {'Total': 'United States'})
state_stations_df.to_csv('state_stations.csv')
state_stations_df.head(53)

,Number_of_Stations_2016,Number_of_Outlets_2016,Number_of_Stations_2017,Number_of_Outlets_2017,Number_of_Stations_2018,Number_of_Outlets_2018,Number_of_Stations_2019,Number_of_Outlets_2019,Number_of_Stations_2020,Number_of_Outlets_2020,Number_of_Stations_2021,Number_of_Outlets_2021
State,,,,,,,,,,,,
Alabama,113,227,146,320,165,395,195,506,223,596,276,680
Alaska,3,5,7,11,7,11,19,34,25,45,52,94
Arizona,424,1016,432,1066,479,1226,522,1476,574,1778,890,2257
Arkansas,63,91,62,113,70,124,98,290,112,350,160,453
California,4175,14084,4657,16538,5469,21119,6435,27128,7671,34622,14616,41225
Colorado,443,990,555,1383,709,1792,861,2516,1062,3234,1614,3978
Connecticut,339,723,355,788,383,837,422,1048,456,1240,533,1430
Delaware,30,67,37,102,50,133,57,172,70,206,134,314
District of Columbia,101,242,96,259,123,333,172,553,190,701,289,822


In [22]:
def make_json(csv_file_path, json_file_path):
    data = {}
    with open(csv_file_path, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        i = 1
        for rows in csvReader:
#             key = rows['State']
            data [i]=rows
            i = i+1
    with open (json_file_path, 'w', encoding = 'utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
csv_file_path = r'state_stations.csv'
json_file_path = r'state_stations.json'

make_json(csv_file_path, json_file_path)

In [26]:
with open ('data.json', 'r') as file:
    data1 = json.load(file)
with open ('state_stations.json', 'r') as file:
    data2 = json.load(file)
merged_data = {}
for key in data1:
    if key in data2:
        merged_data[key] = {**data1[key], **data2[key]}
with open ('combined_data.json', 'w') as file:
    json.dump(merged_data, file)
